In [19]:
import ee
from functools import partial

In [3]:
ee.Initialize()

In [4]:
YEAR_TO_MAP = 2002

In [5]:
_years = ee.List.sequence(YEAR_TO_MAP-1, YEAR_TO_MAP)


_start_date = ee.Date.fromYMD(YEAR_TO_MAP-1, 1, 1)
_end_date = ee.Date.fromYMD(YEAR_TO_MAP, 12, 31)

In [6]:
mod09a1_full = ee.ImageCollection("MODIS/006/MOD09A1")
mod09a1 = mod09a1_full.filterDate(_start_date, _end_date)
MODIS_COLLECTION=mod09a1
MODIS_LANDCOVER = ee.ImageCollection("MODIS/006/MCD12Q1")

Get the various dates that are needed to filter the imagery for the different steps

The rice calculation algorithm needs input images for 40 days before the first date for which output is required, and for each input image we need to have the corresponding full calendar year's worth of data to compute the masking. So, to complete the rice map for a given year, we also need the previous year's data.

### MOD09A1 NORMALISED INDEX COMPUTATION FUNCTIONS 
Define functions to compute the various normalised indices used in the algorithm, on a single
MOD09A1 image.

In [7]:
def unScale(image):
    return image.float().divide(10000)

def computeNDVI(image):
    nir = unScale(image.select('sur_refl_b02'))
    red = unScale(image.select('sur_refl_b01'))
    ndvi = (nir.subtract(red)).divide(nir.add(red))
    return ndvi.rename('ndvi').set('system:time_start', image.get('system:time_start'))

def computeLSWI(image):
    nir = unScale(image.select('sur_refl_b02'))
    swir = unScale(image.select('sur_refl_b06'))
    lswi = (nir.subtract(swir)).divide(nir.add(swir))
    return lswi.rename('lswi').set('system:time_start', image.get('system:time_start'))

def computeEVI(image):
    nir = unScale(image.select('sur_refl_b02'))
    red = unScale(image.select('sur_refl_b01'))
    blue = unScale(image.select('sur_refl_b03'))
    evi = (nir.subtract(red)
        .divide(nir.add(red.multiply(6.0)).subtract(blue.multiply(7.5)).add(1.0))
        .multiply(2.5)
        .clamp(0.0, 1.0))
    return evi.rename('evi').set('system:time_start', image.get('system:time_start'))

def computeNDSI(image):
    # the referenced paper used nir band but most other references use swir
    # for example doi:10.1016/j.rse.2003.10.016
    # Doing it with nir and a threshold of snow = 0.4+ gives completely nonsense results
    # so i assume this was a misprint
    nir = unScale(image.select('sur_refl_b02'))
    swir = unScale(image.select('sur_refl_b06'))
    green = unScale(image.select('sur_refl_b04'))
    ndsi = (green.subtract(swir)).divide(green.add(swir))
    return ndsi.rename('ndsi').set('system:time_start', image.get('system:time_start'))


### ALGORITHM STEPS EXTRACTED FROM NARRATIVE OF THE PAPER

Steps are numbered according to how they appear in the text but are not executed in this order.

#### **#1.** Compute flooded areas 

Defined as pixels where LSWI is nearly as high, or higher, than NDVI or EVI


In [8]:
def computeFlooded(image):
    lswi = computeLSWI(image)
    evi = computeEVI(image)
    ndvi = computeNDVI(image)
    test = evi.min(ndvi)
    flood = lswi.add(0.05).gte(test)
    return flood.rename('flood').set('system:time_start', image.get('system:time_start'))

### **#2.** Define a procedure to determine whether rice growth occurs in (a) pixel

The algorithm is based on the assumption that the EVI value of a true rice pixel reaches half the maximum EVI value (in that crop cycle) within 5 x 8-day composites (40 days) following the date of flooding and transplanting.

This needs breaking down into several steps:

****************** These stages are implemented further below *****************

**2. a.** Identify dates when flooding occurs (relative to a previous image that was not flooded)

**2. b.** Identify maximum EVI values reached by each pixel in each crop cycle (or, let's say ever)

**2. c.** Identify locations / times where the EVI was at least half of this maximum value

**2. d.** Identify from these locations/times the ones which are within 40 days of a flooding date


### **#3.** Cloud cover mask

"*Exclude pixels where marked as cloud in the QC band, and also exclude pixels where blue band 
reflectance GTE 0.2*"

This description is somewhat inconclusive of what they actually did. From the MODIS doc: 
    
    ```bits 0-1 are a cloud mask read from MOD35 and bit 10 is a cloud mask generated by PGE11's
    internal cloud algorithm```
    
Xiao et al does not specify which of these masks was used so we'll extract each mask from the data 
(using bitwise operators) and mask where either is true (boolean OR)
   


In [9]:

def computeCloudOrBlueMask(image):
    """ Identify pixels where either MODIS cloud mask is set OR blue band >= 0.2

    NB returns 0 where cloudy and 1 otherwise, for use with updateMask"""
    blue = unScale(image.select('sur_refl_b03'))
    # from MODIS doc: "All cloud information should be derived from State QA SDSs" (not the band qa)
    # So we select that band and for clarity separate it into two separate mask images
    qa = image.select('StateQA')
    # bits 0-1 are one of the recorded cloud masks with the following values: 
    # 00 = clear, 01 = cloudy, 10 = mixed, 11 = unset, assumed clear
    cloudMOD35 = (qa.bitwise_and(3).eq(1)).Or(qa.bitwise_and(3).eq(1))
    # bit 10 is the other recorded cloud mask with key: 0 = clear, 1 = cloud
    cloudInternal = qa.bitwise_and(1024).eq(1024)
    return (cloudMOD35
        .Or(cloudInternal)
        .Not()
        # turned the blue check off for the time being as it doesn't appear necessary and it marks clouds 
        # in the middle of clear deserts
        .Or(blue.gte(0.2)) 
        .rename('cloud').set('system_time_start', image.get('system:time_start')))

def computeCloudMODISMask(image):
    qa = image.select('StateQA');
    # from MODIS doc: "bits 0-1 are a cloud mask read from MOD35"
    # bits 0-1 key: 00 = clear, 01 = cloudy, 10 = mixed, 11 = unset, assumed clear
    cloudMOD35 = (qa.bitwise_and(3).eq(1)).Or(qa.bitwise_and(3).eq(1));
    return (cloudMOD35
        .Not()
        .rename('cloud').set('system_time_start', image.get('system:time_start')))

def computeCloudInternalMask(image):
    qa = image.select('StateQA')
    # from MODIS doc: "bit 10 is a cloud mask generated by PGE11's internal cloud algorithm"
    # bit 10 key: 0 = clear, 1 = cloud
    cloudInternal = qa.bitwise_and(1024).eq(1024)
    return (cloudInternal
        .Not()
        .rename('cloud').set('system_time_start', image.get('system:time_start')))


### **#4.** Snow mask

"*Filter snow pixels based on NDSI and NIR reflectance, where NDSI > 0.40 and NIR > 0.11"

NB NDSI has been implemented using SWIR band, not NIR, but Xiao et al state NIR - this appears 
wrong. Unsure if we should also use SWIR in the additional check here.

NB - Returns 0 where snow and 1 otherwise, for use with updateMask

In [10]:
def compute_snow_mask(image):
    ndsi = computeNDSI(image)
    nir = image.select('sur_refl_b02')
    snow = ndsi.gte(0.40).bitwise_and(nir.gte(0.11)).Not()
    return snow.rename('snow').set('system:time_start', image.get('system:time_start'))

### **5. a.** Identify water areas 

*Flag pixels as water where NDVI < 0.10 and NDVI < LSWI*

It's unclear from the paper why they don't simply use the "flooded" function that was defined in step 1, rather than redefining a slightly different version that doesn't use EVI.

In [11]:
def compute_water(image):
    lswi = computeLSWI(image)
    ndvi = computeNDVI(image)
    water = ndvi.lt(0.1).And(ndvi.lt(lswi))
    return water.rename('water').set('system:time_start', image.get('system:time_start'))

Define static masks based on elevation, slope and ocean

In [12]:
elev_image = ee.Image("USGS/GTOPO30")
elev_mask = elev_image.lte(2000)

slope_mask = ee.Terrain.slope(elev_image).lte(3)

def get_landcover_water_mask(year):
    modis_landcover_coll = ee.ImageCollection("MODIS/006/MCD12Q1")
    modis_lc_image = (modis_landcover_coll.filter(ee.Filter.calendarRange(
                     year, year, 'year'))
                     .first())
    igbp_image = modis_lc_image.select('LC_Type1')
    ocean_mask = igbp_image.neq(17)
    return ocean_mask

ocean_mask = get_landcover_water_mask(YEAR_TO_MAP)

In [15]:
def apply_all_modis_masks(img):
    cloud_mask_internal = computeCloudInternalMask(img) # 1 = clear 0 = cloud
    cloud_mask_modis = computeCloudMODISMask(img) # 1 = clear 0 = cloud
    # 0 = cloud, only when both masks say cloud, slightly confusing negation
    cloud_mask_both = cloud_mask_internal.Or(cloud_mask_modis) 
    snow_mask = compute_snow_mask(img)
    return (img.set('year', ee.Date(img.get('system:time_start')).get('year'))
        .updateMask(ocean_mask)
        .updateMask(elev_mask)
        .updateMask(slope_mask)
        .updateMask(cloud_mask_both)
        .updateMask(snow_mask))


In [16]:
masked_modis_series = MODIS_COLLECTION.map(apply_all_modis_masks)

## Define time-series analysis functions

The functions defined so far each operate on a single image at once. Now we define functions that rely on the image series.

### **5. b.** Permanent water mask

*Identify pixels covered by water, then exclude from the analysis those pixels which are water for more than 10 images (8 day blocks) i.e. 80 days per year*

Any pixels identified as water covered for mor than this will be treated as permanent water and not analysed for the wetting/drying cycle that triggers rice detection. 

I've modified this to also flag pixels which are water for more than 30% of the images, in case there are many missing values and the hardcoded 10-occurrence threshold isn't valid.

Note that the .map() function needs a callable which takes one argument. To map over a list of years for a specified ImageCollection we need to pass both the ImageCollection and a reference to the year in question. We use `functools.partial` to convert the two-argument function and its first argument (the imagecollection) into a second function taking only one argument (the year), by creatin a closure. This can be used within an EE map, over the years.

In [21]:
def get_annual_permanent_water(collection, year):
    """Returns a mask image to select pixels which are water in >10 images AND >30% of them"""
    start = ee.Date.fromYMD(year, 1, 1)
    # this approach enables us to use non-calendar years e.g. april-april or whatever
    stop = start.advance(0, 'year').advance(12, 'month').advance(0,'day')
    waterYear = (collection.filter(ee.Filter.date(start, stop))
        .map(compute_water))
    waterYearCount = waterYear.sum()
    waterYearMean = waterYear.mean()
    # let's also try to implement a threshold for the average because the count will vary 
    w = waterYearCount.gt(10).And(waterYearMean.gt(0.3))
    return (w.set('year', year)
        .set('system:time_end',ee.Date.fromYMD(year, 1, 1).advance(1, 'year'))
        .set('system:time_start',ee.Date.fromYMD(year, 1, 1))
        .Not())

In [22]:
year_watermask_mapper = partial(get_annual_permanent_water, masked_modis_series)

permanent_water_mask_by_year = ee.ImageCollection.fromImages(
    _years.map(year_watermask_mapper).flatten())

### **6.** Mask out evergreen vegetation

**6. a.** Identify evergreen forest pixels using the algorithm:

    "*Identify areas where NDVI > 0.7 for at least 20 8-day composites per year, to flag evergreen forests*"


In [23]:
def get_year_is_evergreen(collection, year):
    start = ee.Date.fromYMD(year, 1, 1)
    stop = start.advance(12, 'month').advance(0, 'day')
    ndviGreenYearColl = (collection.filter(ee.Filter.date(start, stop))
          .map(lambda img: computeNDVI(img).gt(0.7))
    )
    ndviGreenYearCount = ndviGreenYearColl.sum()
    # ndviGreenYearMean = ndviGreenYearColl.mean()
    # TODO - maybe a mean value too if n is say between 10 and 20? 
    forestYear = ndviGreenYearCount.gt(20).rename("maybeForest")
    return (forestYear.set('year', year)
        .set('system:time_end',ee.Date.fromYMD(year, 1, 1).advance(1, 'year'))
        .set('system:time_start',ee.Date.fromYMD(year, 1, 1)))
    

In [24]:
year_evergreen_mapper = partial(get_year_is_evergreen, masked_modis_series)
evergreen_forest_by_year = ee.ImageCollection.fromImages(
    _years.map(year_evergreen_mapper).flatten())

**6. b.** Identify non-forest evergreen pixels using the algorithm:

    "*Identify areas where LSWI is always GTE 0.10, to flag natural evergreen (non forest) vegetation*"
    
NB: This has been tuned by the authors for the original study area; it may not be globally suitable!


In [25]:
def get_year_is_evergreen_nonforest(collection, year):
    start = ee.Date.fromYMD(year, 1, 1)
    stop = start.advance(12, 'month').advance(0, 'day')
    shrubby_year_img = (collection.filter(ee.Filter.date(start, stop))
          .map(lambda img: computeLSWI(img).gte(0.1))
    ).reduce(ee.Reducer.allNonZero()).rename("maybeShrubbery")
    return (shrubby_year_img.set('year', year)
        .set('system:time_end',ee.Date.fromYMD(year, 1, 1).advance(1, 'year'))
        .set('system:time_start',ee.Date.fromYMD(year, 1, 1)))
    

In [26]:
year_shrubbery_mapper = partial(get_year_is_evergreen_nonforest, masked_modis_series)
shrubbery_by_year = ee.ImageCollection.fromImages(
    _years.map(year_shrubbery_mapper).flatten())

Combine the two types of evergreen estimate into a single image per year

In [27]:
def combine_evergreen_estimates(image):
    forest_band = image.select("maybeForest")
    shrubbery_band = image.select("maybeShrubbery")
    return (forest_band.Or(shrubbery_band)
            .rename("evergreen")
            .set('system:time_end',image.get('system:time_end'))
            .set('system:time_start',image.get('system:time_start'))
            .set('year', image.get('year'))
            .Not())

evergreen_mask_by_year = (evergreen_forest_by_year
                         .combine(shrubbery_by_year)
                         .map(lambda img: combine_evergreen_estimates(img)))

**6. c.** Combine all the annual masks into one image per year

For each year, combine the evergreen and permanent water masks derived above into a single 2-band image, and make them into a collection with one such image per year

In [28]:
annual_masks = permanent_water_mask_by_year.combine(evergreen_mask_by_year)

**6. d.** Extra step (not in paper): make a forest mask from the MODIS landcover data

For each annual landcover (MCD12Q1) image, select the IGBP classes representing forest and use these as an additional masking step as well as the forest estimates based on reflectance indices above

In [29]:
def get_mcd12q1_forest(image):
    igbp = image.select('LC_Type1')
    # igbp evergreen forest classes 1, 2; deciduous classes 3, 4, mixed forest class 5
    year = ee.Date(image.get('system:time_start')).get('year')
    igbpForest = igbp.eq(1).Or(igbp.eq(2)).Or(igbp.eq(3)).Or(igbp.eq(4)).Or(igbp.eq(5))
    return (igbpForest
        .rename('evergreen_landcover')
        .set('system:time_start',image.get('system:time_start'))
        .set('year', year)
        .Not())
annual_mcd12q1_forest = MODIS_LANDCOVER.map(get_mcd12q1_forest)

Because this comes from a different dataset to the other masks, the images have different IDs and so we cannot use the .combine() function to join them into a single mask dataset.

Instead, we define a join that will combine them based on the year property:

In [30]:
get_annual_LC_join = ee.Join.saveBest(**{
  "matchKey": "lcImage",
  "measureKey": "yearDiff"
})

landcover_join_result = get_annual_LC_join.apply(**{
  "primary": annual_masks,
  "secondary": annual_mcd12q1_forest,
  "condition": ee.Filter.maxDifference(**{
    "difference": 5, # allow some offset as we may have more recent rice images than landcover ones
    "leftField": "year",
    "rightField": "year"
  })
})

# The join saves the matched LC image as a property on the incoming image, not as a band.
# We need to transfer it to a band:
def extract_lc_joinresult(image):
    lc_image = ee.Image(image.get('lcImage'))
    image = ee.Image(image).addBands(lc_image)
    return image
    
annual_masks = landcover_join_result.map(lambda img: extract_lc_joinresult(img))

## Begin main processing

Now we have defined all the masking and image-processing functions, we can use them to process the surface reflectance data and then implement the core rice-detection algorithm.

First we apply all the masking to the incoming SR data. 

### 1. a. For each image in the incoming filtered MODIS surface reflectance dataset, apply the cloud/snow mask based on that image

In [87]:
...

Ellipsis

### 1. b. For each image in the resultant collection, apply the water/evergreen mask for the corresponding year

Join based on the year property; extract the join results from properties to bands, and then update the image's mask accordingly.

In [31]:
get_annual_mask_join = ee.Join.inner()
join_res = get_annual_mask_join.apply(**{
  "primary": masked_modis_series,
  "secondary": annual_masks,
  "condition": ee.Filter.equals(**{
    "leftField": "year",
    "rightField": "year"
  })
})

def extract_mask_join_results(feature):
    image = ee.Image(feature.get('primary'));
    annual_mask = ee.Image(feature.get('secondary'));
    masked = (image
        .updateMask(annual_mask.select('water'))
        .updateMask(annual_mask.select('evergreen'))
        .updateMask(annual_mask.select('evergreen_landcover')))
    return masked; 

fully_masked_series = ee.ImageCollection(
    join_res.map(lambda img: extract_mask_join_results(img)))


### 2. Implement rice detection

As a reminder the core principle of the algorithm is to track (on a pixel-wise basis) the point at which flooding occurs, and the point at which greening occurs. Where greening occurs within 40 days after flooding, and all the masking conditions are met, this is considered to be an indicator of paddy rice.

### 2. a. Identify dates when flooding onset occurs

That is, in each image find pixels which are flooded but were not flooded in the previous image

In [32]:
# Get a series of binary images showing flooded pixels
flooded_images = fully_masked_series.map(
    lambda image: computeFlooded(image))

# join each is-a-flood image to the previous one temporally
save_prev_flood_join = ee.Join.saveFirst(**{
  "matchKey": "prevFlood",
  "ordering": "system:time_start",
  "ascending": False
})

floods_with_prev_img = save_prev_flood_join.apply(**{
  "primary": flooded_images,
  "secondary": flooded_images,
  "condition": ee.Filter.greaterThan(**{
    "leftField": "system:time_start",
    "rightField": "system:time_start"
  })
})

# build a collection of images that are 1 where a pixel is flooded now but wasn't in previous image

def extract_flood_join_results(joined_image):
    prev_image = ee.Image(joined_image.get('prevFlood'))
    this_image = ee.Image(joined_image)
    is_new_flood = this_image.And(prev_image.Not())
    return (is_new_flood.set('system:time_start', 
                          this_image.get('system:time_start'))
            .rename('flood-starts'))

flood_initiations = floods_with_prev_img.map(extract_flood_join_results)
flood_initiations = ee.ImageCollection(flood_initiations)

### 2. b. Identify maximum EVI values reached by each pixel

For now, we're doing this as max _ever_ reached in each pixel over the whole series, rather than per crop cycle

In [33]:
max_evi_threshold = fully_masked_series.map(
    lambda image: computeEVI(image)
).max().divide(2.0)

### 2. c. Identify pixels where EVI was at least 50% of this maximum value

In each image, calculate the EVI and mark the locations where it is at least 50% of the highest EVI value ever recorded in that pixel.

At the same time (for convenience) we calculate a DateRange representing the 40 days prior to the date of each image, and attach it to the image.

In [34]:
def evi_over_threshold(threshold_image, test_image):
    forty_days_ago = (ee.Date(test_image.get('system:time_start'))
                      .advance(-40, 'day'))
    forty_day_window = ee.DateRange(forty_days_ago, 
                                    test_image.get('system:time_start'))
    return (computeEVI(test_image)
            .gt(threshold_image)
            .rename('high-evi')
            .set('system:time_start', 
                 test_image.get('system:time_start'))
            .set('forty_day_window', forty_day_window))

evi_over_threshold_mapper = partial(evi_over_threshold, max_evi_threshold)
evi_over_threshold_dates = fully_masked_series.map(evi_over_threshold_mapper)


### 2. d. Identify from the high-EVI pixels those which are within 40 days of a flooding event

We do this by joining the high-EVI images to the flood images, based on a key of the forty-day window we calculated.

This will provide our output rice estimate map.

In [35]:
# This join needs to look at more than one preceding image so use a saveAll not saveBest
save_flood_onsets_join = ee.Join.saveAll(**{
  "matchesKey": 'floodsInLast40D',
  "ordering": 'system:time_start',
  "ascending": False
})

green_with_recent_floods = save_flood_onsets_join.apply(**{
    "primary": evi_over_threshold_dates,
    "secondary": flood_initiations,
    "condition": ee.Filter.dateRangeContains(**{
        "leftField": "forty_day_window",
        "rightField": "system:time_start"
  })
})

In [36]:
def extract_rice_join_results(image):
    # in the saveAll join, the whole matching set of images are saved onto the 
    # specified property of each image
    recent_floods_coll = ee.ImageCollection.fromImages(image.get('floodsInLast40D'))
    any_recent_floods = recent_floods_coll.reduce(ee.Reducer.anyNonZero())
    is_it_green = ee.Image(image)
    is_it_rice = is_it_green.And(any_recent_floods);
    return (is_it_rice.rename('rice')
        .set('system:time_start', image.get('system:time_start')))

rice_image_collection = green_with_recent_floods.map(extract_rice_join_results)
rice_image_collection = ee.ImageCollection(rice_image_collection)

In [37]:
rice_in_year = ee.Image(rice_image_collection
                        .filterDate(ee.Date.fromYMD(
                            YEAR_TO_MAP, 1, 1), ee.Date.fromYMD(YEAR_TO_MAP, 12, 31))
                       .reduce(ee.Reducer.anyNonZero()))

In [38]:
print(rice_in_year.getInfo())

{'type': 'Image', 'bands': [{'id': 'rice_any', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [43]:
rice_in_year = rice_in_year.updateMask(rice_in_year)

In [47]:
import folium

In [50]:
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)
folium.Map.add_ee_layer = add_ee_layer

In [54]:
rice_map = folium.Map(location=[0, 120], zoom_start=3, height=500)
rice_map.add_ee_layer(rice_in_year, {"palette":['000000', 'FF0000']}, "RICE")

In [55]:
rice_map.add_child(folium.LayerControl())

In [42]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…